In [5]:
import tensorflow as tf
from utils.data import *
from utils.lstm_rnn import *
import os
import pandas as pd
if not os.path.isdir("save"):
    os.mkdir("save")
##### config #####
save_dataset = False # 儲存dataset，true: 生成dataset、false: 使用save/ 儲存的
##### config #####

In [7]:

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], enable=True)

repeat = 100
if save_dataset:
    X_raw, y = generate_dataset(repeat=repeat)
    X = tf.reshape(X_raw, [y.shape[0],3,60])
    np.save("save/X_raw.npy", X_raw.numpy())
    np.save("save/y.npy", y)
else:
    X_raw = np.load("save/X_raw.npy")
    y = np.load("save/y.npy")

X_raw = tf.convert_to_tensor(X_raw)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [10]:
try_timesteps=[3,6]#list(range(3,18,3))
try_units=[100]#[50,100,200,300,180]#list(range(10,200,20))
try_epochs=[150]

"""
try_timesteps=[3]
try_units=[180]#list(range(10,200,20))
try_epochs=[150]
"""
df_all = pd.DataFrame([], columns=[["loss", "out_1_loss", "out_2_loss", "out_1_acc", "out_2_acc", "timesteps", "units", "epoch"]])
for timesteps in try_timesteps:
    for units in try_units:
        for epoch in try_epochs:
            X = tf.reshape(X_raw, [y.shape[0],timesteps,180//timesteps])
            train_dataset, val_dataset, test_dataset = split_dataset(X, y)
            
            model = construct_model(timesteps=timesteps, data_dim = 180//timesteps, units=units)
            model, history = compile_fit_model(model,epochs=epoch,train_dataset=train_dataset,val_dataset=val_dataset)
            model.save("saved_model/lstm-rnn-unit("+str(units)+")-timesteps("+str(timesteps)+")-epoch("+str(epoch)+")")
            results = model.evaluate(test_dataset)
            results.extend([timesteps, units, epoch])
            df = pd.DataFrame([results], columns=[["loss", "out_1_loss", "out_2_loss", "out_1_acc", "out_2_acc", "timesteps", "units", "epoch"]])
            df_all = df_all.append(df)
df_all.to_csv("save/result.csv")

112000 24000 24000
train: 112000
val: 24000
test: 24000
new train 112000
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 3, 60)]      0                                            
__________________________________________________________________________________________________
vfc_1_1 (LSTM)                  [(None, 3, 100), (No 64400       input_8[0][0]                    
__________________________________________________________________________________________________
vfc_1_2 (LSTM)                  [(None, 100), (None, 80400       vfc_1_1[0][0]                    
__________________________________________________________________________________________________
vfc_2_1 (LSTM)                  (None, 3, 100)       64400       input_8[0][0]                    
                   

28/28 [==============================] - 1s 25ms/step - loss: 0.1864 - out_1_loss: 0.1047 - out_2_loss: 0.0817 - out_1_acc: 0.9422 - out_2_acc: 0.9549 - val_loss: 0.2012 - val_out_1_loss: 0.1113 - val_out_2_loss: 0.0899 - val_out_1_acc: 0.9382 - val_out_2_acc: 0.9514
Epoch 20/150
28/28 [==============================] - 1s 25ms/step - loss: 0.1850 - out_1_loss: 0.1037 - out_2_loss: 0.0813 - out_1_acc: 0.9424 - out_2_acc: 0.9551 - val_loss: 0.2003 - val_out_1_loss: 0.1105 - val_out_2_loss: 0.0898 - val_out_1_acc: 0.9382 - val_out_2_acc: 0.9517
Epoch 21/150
28/28 [==============================] - 1s 26ms/step - loss: 0.1838 - out_1_loss: 0.1028 - out_2_loss: 0.0809 - out_1_acc: 0.9426 - out_2_acc: 0.9549 - val_loss: 0.1997 - val_out_1_loss: 0.1099 - val_out_2_loss: 0.0897 - val_out_1_acc: 0.9385 - val_out_2_acc: 0.9516
Epoch 22/150
28/28 [==============================] - 1s 26ms/step - loss: 0.1826 - out_1_loss: 0.1021 - out_2_loss: 0.0806 - out_1_acc: 0.9428 - out_2_acc: 0.9553 - val_

28/28 [==============================] - 1s 25ms/step - loss: 0.1573 - out_1_loss: 0.0909 - out_2_loss: 0.0663 - out_1_acc: 0.9484 - out_2_acc: 0.9633 - val_loss: 0.2061 - val_out_1_loss: 0.1089 - val_out_2_loss: 0.0973 - val_out_1_acc: 0.9390 - val_out_2_acc: 0.9526
Epoch 49/150
28/28 [==============================] - 1s 25ms/step - loss: 0.1560 - out_1_loss: 0.0906 - out_2_loss: 0.0655 - out_1_acc: 0.9487 - out_2_acc: 0.9638 - val_loss: 0.2071 - val_out_1_loss: 0.1091 - val_out_2_loss: 0.0980 - val_out_1_acc: 0.9393 - val_out_2_acc: 0.9515
Epoch 50/150
28/28 [==============================] - 1s 26ms/step - loss: 0.1549 - out_1_loss: 0.0902 - out_2_loss: 0.0647 - out_1_acc: 0.9489 - out_2_acc: 0.9646 - val_loss: 0.2079 - val_out_1_loss: 0.1094 - val_out_2_loss: 0.0985 - val_out_1_acc: 0.9391 - val_out_2_acc: 0.9514
Epoch 51/150
28/28 [==============================] - 1s 26ms/step - loss: 0.1536 - out_1_loss: 0.0898 - out_2_loss: 0.0638 - out_1_acc: 0.9492 - out_2_acc: 0.9647 - val_

28/28 [==============================] - 1s 25ms/step - loss: 0.1246 - out_1_loss: 0.0773 - out_2_loss: 0.0474 - out_1_acc: 0.9577 - out_2_acc: 0.9734 - val_loss: 0.2522 - val_out_1_loss: 0.1194 - val_out_2_loss: 0.1327 - val_out_1_acc: 0.9400 - val_out_2_acc: 0.9525
Epoch 78/150
28/28 [==============================] - 1s 30ms/step - loss: 0.1234 - out_1_loss: 0.0768 - out_2_loss: 0.0466 - out_1_acc: 0.9581 - out_2_acc: 0.9740 - val_loss: 0.2537 - val_out_1_loss: 0.1200 - val_out_2_loss: 0.1337 - val_out_1_acc: 0.9402 - val_out_2_acc: 0.9527
Epoch 79/150
28/28 [==============================] - 1s 30ms/step - loss: 0.1230 - out_1_loss: 0.0762 - out_2_loss: 0.0467 - out_1_acc: 0.9583 - out_2_acc: 0.9738 - val_loss: 0.2559 - val_out_1_loss: 0.1207 - val_out_2_loss: 0.1352 - val_out_1_acc: 0.9398 - val_out_2_acc: 0.9517
Epoch 80/150
28/28 [==============================] - 1s 25ms/step - loss: 0.1221 - out_1_loss: 0.0758 - out_2_loss: 0.0463 - out_1_acc: 0.9588 - out_2_acc: 0.9738 - val_

28/28 [==============================] - 1s 26ms/step - loss: 0.1028 - out_1_loss: 0.0643 - out_2_loss: 0.0385 - out_1_acc: 0.9663 - out_2_acc: 0.9799 - val_loss: 0.3080 - val_out_1_loss: 0.1393 - val_out_2_loss: 0.1687 - val_out_1_acc: 0.9414 - val_out_2_acc: 0.9524
Epoch 107/150
28/28 [==============================] - 1s 26ms/step - loss: 0.1019 - out_1_loss: 0.0639 - out_2_loss: 0.0380 - out_1_acc: 0.9667 - out_2_acc: 0.9803 - val_loss: 0.3094 - val_out_1_loss: 0.1405 - val_out_2_loss: 0.1690 - val_out_1_acc: 0.9412 - val_out_2_acc: 0.9527
Epoch 108/150
28/28 [==============================] - 1s 25ms/step - loss: 0.1007 - out_1_loss: 0.0633 - out_2_loss: 0.0374 - out_1_acc: 0.9670 - out_2_acc: 0.9808 - val_loss: 0.3114 - val_out_1_loss: 0.1410 - val_out_2_loss: 0.1704 - val_out_1_acc: 0.9410 - val_out_2_acc: 0.9520
Epoch 109/150
28/28 [==============================] - 1s 25ms/step - loss: 0.1011 - out_1_loss: 0.0629 - out_2_loss: 0.0382 - out_1_acc: 0.9674 - out_2_acc: 0.9802 - v

...

28/28 [==============================] - 1s 31ms/step - loss: 0.1704 - out_1_loss: 0.0961 - out_2_loss: 0.0743 - out_1_acc: 0.9456 - out_2_acc: 0.9592 - val_loss: 0.1980 - val_out_1_loss: 0.1079 - val_out_2_loss: 0.0901 - val_out_1_acc: 0.9396 - val_out_2_acc: 0.9527
Epoch 33/150
28/28 [==============================] - 1s 31ms/step - loss: 0.1698 - out_1_loss: 0.0958 - out_2_loss: 0.0740 - out_1_acc: 0.9459 - out_2_acc: 0.9596 - val_loss: 0.1983 - val_out_1_loss: 0.1080 - val_out_2_loss: 0.0903 - val_out_1_acc: 0.9395 - val_out_2_acc: 0.9530
Epoch 34/150
28/28 [==============================] - 1s 37ms/step - loss: 0.1692 - out_1_loss: 0.0956 - out_2_loss: 0.0736 - out_1_acc: 0.9458 - out_2_acc: 0.9600 - val_loss: 0.1992 - val_out_1_loss: 0.1080 - val_out_2_loss: 0.0912 - val_out_1_acc: 0.9402 - val_out_2_acc: 0.9521
Epoch 35/150
28/28 [==============================] - 1s 36ms/step - loss: 0.1686 - out_1_loss: 0.0953 - out_2_loss: 0.0733 - out_1_acc: 0.9457 - out_2_acc: 0.9602 - val_

28/28 [==============================] - 1s 32ms/step - loss: 0.1520 - out_1_loss: 0.0888 - out_2_loss: 0.0632 - out_1_acc: 0.9498 - out_2_acc: 0.9650 - val_loss: 0.2158 - val_out_1_loss: 0.1104 - val_out_2_loss: 0.1054 - val_out_1_acc: 0.9416 - val_out_2_acc: 0.9533
Epoch 62/150
28/28 [==============================] - 1s 38ms/step - loss: 0.1513 - out_1_loss: 0.0885 - out_2_loss: 0.0628 - out_1_acc: 0.9505 - out_2_acc: 0.9650 - val_loss: 0.2153 - val_out_1_loss: 0.1102 - val_out_2_loss: 0.1051 - val_out_1_acc: 0.9414 - val_out_2_acc: 0.9527
Epoch 63/150
28/28 [==============================] - 1s 32ms/step - loss: 0.1504 - out_1_loss: 0.0882 - out_2_loss: 0.0621 - out_1_acc: 0.9509 - out_2_acc: 0.9655 - val_loss: 0.2137 - val_out_1_loss: 0.1099 - val_out_2_loss: 0.1038 - val_out_1_acc: 0.9411 - val_out_2_acc: 0.9521
Epoch 64/150
28/28 [==============================] - 1s 32ms/step - loss: 0.1498 - out_1_loss: 0.0880 - out_2_loss: 0.0618 - out_1_acc: 0.9508 - out_2_acc: 0.9656 - val_

28/28 [==============================] - 1s 32ms/step - loss: 0.1291 - out_1_loss: 0.0787 - out_2_loss: 0.0504 - out_1_acc: 0.9564 - out_2_acc: 0.9714 - val_loss: 0.2448 - val_out_1_loss: 0.1191 - val_out_2_loss: 0.1257 - val_out_1_acc: 0.9397 - val_out_2_acc: 0.9522
Epoch 91/150
28/28 [==============================] - 1s 31ms/step - loss: 0.1292 - out_1_loss: 0.0785 - out_2_loss: 0.0507 - out_1_acc: 0.9565 - out_2_acc: 0.9713 - val_loss: 0.2463 - val_out_1_loss: 0.1203 - val_out_2_loss: 0.1260 - val_out_1_acc: 0.9394 - val_out_2_acc: 0.9521
Epoch 92/150
28/28 [==============================] - 1s 31ms/step - loss: 0.1282 - out_1_loss: 0.0782 - out_2_loss: 0.0500 - out_1_acc: 0.9567 - out_2_acc: 0.9716 - val_loss: 0.2473 - val_out_1_loss: 0.1198 - val_out_2_loss: 0.1274 - val_out_1_acc: 0.9395 - val_out_2_acc: 0.9523
Epoch 93/150
28/28 [==============================] - 1s 31ms/step - loss: 0.1274 - out_1_loss: 0.0777 - out_2_loss: 0.0497 - out_1_acc: 0.9572 - out_2_acc: 0.9721 - val_

28/28 [==============================] - 1s 32ms/step - loss: 0.1143 - out_1_loss: 0.0696 - out_2_loss: 0.0447 - out_1_acc: 0.9620 - out_2_acc: 0.9753 - val_loss: 0.2851 - val_out_1_loss: 0.1318 - val_out_2_loss: 0.1533 - val_out_1_acc: 0.9408 - val_out_2_acc: 0.9514
Epoch 120/150
28/28 [==============================] - 1s 32ms/step - loss: 0.1144 - out_1_loss: 0.0695 - out_2_loss: 0.0449 - out_1_acc: 0.9619 - out_2_acc: 0.9751 - val_loss: 0.2867 - val_out_1_loss: 0.1328 - val_out_2_loss: 0.1539 - val_out_1_acc: 0.9399 - val_out_2_acc: 0.9518
Epoch 121/150
28/28 [==============================] - 1s 32ms/step - loss: 0.1144 - out_1_loss: 0.0695 - out_2_loss: 0.0449 - out_1_acc: 0.9624 - out_2_acc: 0.9753 - val_loss: 0.2874 - val_out_1_loss: 0.1352 - val_out_2_loss: 0.1522 - val_out_1_acc: 0.9405 - val_out_2_acc: 0.9522
Epoch 122/150
28/28 [==============================] - 1s 32ms/step - loss: 0.1130 - out_1_loss: 0.0691 - out_2_loss: 0.0439 - out_1_acc: 0.9630 - out_2_acc: 0.9758 - v

28/28 [==============================] - 1s 32ms/step - loss: 0.0982 - out_1_loss: 0.0621 - out_2_loss: 0.0361 - out_1_acc: 0.9677 - out_2_acc: 0.9816 - val_loss: 0.3206 - val_out_1_loss: 0.1508 - val_out_2_loss: 0.1698 - val_out_1_acc: 0.9393 - val_out_2_acc: 0.9517
Epoch 149/150
28/28 [==============================] - 1s 31ms/step - loss: 0.0982 - out_1_loss: 0.0616 - out_2_loss: 0.0366 - out_1_acc: 0.9681 - out_2_acc: 0.9814 - val_loss: 0.3224 - val_out_1_loss: 0.1522 - val_out_2_loss: 0.1702 - val_out_1_acc: 0.9411 - val_out_2_acc: 0.9524
Epoch 150/150
28/28 [==============================] - 1s 33ms/step - loss: 0.0987 - out_1_loss: 0.0617 - out_2_loss: 0.0370 - out_1_acc: 0.9677 - out_2_acc: 0.9812 - val_loss: 0.3221 - val_out_1_loss: 0.1525 - val_out_2_loss: 0.1695 - val_out_1_acc: 0.9412 - val_out_2_acc: 0.9507
INFO:tensorflow:Assets written to: saved_model/lstm-rnn-unit(100)-timesteps(6)-epoch(150)/assets
6/6 [==============================] - 0s 15ms/step - loss: 0.3047 - ou

In [12]:
df_all

,loss,out_1_loss,out_2_loss,out_1_acc,out_2_acc,timesteps,units,epoch
0,0.361949,0.159725,0.202224,0.943208,0.952417,3,100,150
0,0.304710,0.144047,0.160663,0.941167,0.952083,6,100,150


In [11]:
d = df_all.copy().reset_index(drop=True)
_max =  d[["out_1_acc","out_2_acc"]].sum(axis=1).max()
best_result = d[ d[["out_1_acc","out_2_acc"]].sum(axis=1)==_max]
print(best_result)
best_unit = str(best_result.iloc[0]['units'][0])
best_timesteps = str(best_result.iloc[0]['timesteps'][0])
best_epoch = str(best_result.iloc[0]['epoch'][0])

best_model = tf.keras.models.load_model("saved_model/lstm-rnn-unit("+best_unit+")-timesteps("+best_timesteps+")-epoch("+best_epoch+")")
y_pred = best_model.predict(test_dataset)
best_model.evaluate(test_dataset)

       loss out_1_loss out_2_loss out_1_acc out_2_acc timesteps units epoch
0  0.361949   0.159725   0.202224  0.943208  0.952417         3   100   150


ValueError: in user code:

    /home/alanhc-school/miniconda3/envs/tf-ra/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/alanhc-school/miniconda3/envs/tf-ra/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/alanhc-school/miniconda3/envs/tf-ra/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/alanhc-school/miniconda3/envs/tf-ra/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/alanhc-school/miniconda3/envs/tf-ra/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=Fals